In [ ]:
# to allow importing from the parent directory
import sys

# setting path
sys.path.append('../')

### VAE

In [ ]:
from configs.hierarchical_ordered import Config
from utils import load_pretrained_autoencoder
import torch 

cfg = Config()

cfg.data_dir = "../dataset/icons_tensor/"
cfg.meta_filepath = "../dataset/icons_meta.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"

vae_model = load_pretrained_autoencoder(cfg, pretrained_path='../pretrained/hierarchical_ordered.pth.tar', device=device)

# Training Pipeline

In [11]:
import wandb
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dataset.dataset import num_classes, dataloader_with_transformed_dataset
from utils import create_model, train_with_wandb

def model_pipeline(hyperparameters):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # tell wandb to get started
    with wandb.init(project="svgfusion", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # prepare model, optimizer and dataloaders
        train_dataloader, valid_dataloader = dataloader_with_transformed_dataset(vae_model, cfg, 
                                                                                 batch_n=config.batch_size, 
                                                                                 length=config.n_samples)

        model, diffusion = create_model(dropout=config.dropout, predict_xstart=config.predict_xstart,
                                    n_classes=num_classes(train_dataloader), depth=config.depth, 
                                    learn_sigma=config.learn_sigma, num_heads=config.num_heads)

        optimizer = AdamW(model.parameters(), lr=config.learning_rate, weight_decay=0)
        scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
        
        # train the model
        train_with_wandb(model, train_dataloader, diffusion, optimizer, scheduler, config, device=device)

        return model, optimizer, diffusion, scheduler, num_classes(train_dataloader), config

In [ ]:
import wandb

wandb.login()

config = {
        'predict_xstart': True,
        'learn_sigma': True,
        'use_scheduler': True,
        'num_heads': 16,
        'depth': 28,
        'dropout': 0.1,
        'epochs': 100,
        'learning_rate': 0.0001,
        'batch_size': 100,
        'n_samples': None, # None = all of the samples
        'magical_number': 0.7128, # mean of std's of latents
}

model, optimizer, diffusion, scheduler, n_classes, config = model_pipeline(config)

# Saving/Loading the Model

In [ ]:
from utils import save_model

save_model(model, optimizer, diffusion, scheduler, n_classes, config)